In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models.tokenization import *
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from models.model_architecture import *
from models.train_eval import *
from tqdm import tqdm, tqdm_notebook
parent_path='../Data/New_Data_15-06-2020/'

/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/punyajoy/.conda/envs/puny

In [ ]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')


In [ ]:
torch.cuda.set_device(1)

In [4]:
params={'model_path':'bert-base-multilingual-cased',
        'max_length':128,
        'batch_size':6,
        'weights':[1.0,1.0],
        'data_path':parent_path+'Fearspeech_data_final.pkl',
        'max_sentences_per_doc':10,
        'transformer_type':'lstm_transformer',
        'device':'cuda',
        'learning_rate':2e-5,
        'epsilon':1e-8,
        'random_seed':2,
        'epochs':5
       }

In [ ]:
train_phase(params,device = torch.device("cuda"))

In [19]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
    

I0924 09:30:24.217592 140120072415040 connectionpool.py:700] Starting new HTTPS connection (1): s3.amazonaws.com
I0924 09:30:25.342872 140120072415040 connectionpool.py:700] Starting new HTTPS connection (1): s3.amazonaws.com


In [ ]:
temp=tokenizer.tokenize("I am goof")



In [ ]:
temp=['<s>']+temp+['</s>']

In [ ]:
tokenizer.convert_tokens_to_ids(temp)

In [ ]:
from transformers import AutoConfig
config=AutoConfig.from_pretrained('xlm-roberta-base')

In [ ]:
config

In [15]:
annotated_df=pd.read_pickle(params['data_path'])
params_preprocess={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': False}
list_sents=[preprocess_sent(ele,params=params_preprocess).strip() for ele in tqdm_notebook(annotated_df['message_text'],total=len(annotated_df))]

W0924 09:25:16.440173 140120072415040 word.py:92] Falling back to default tokenizer, the NLTK's `TreebankWordTokenizer()`.


In [17]:
import random 
random.sample(list_sents,5)

['रहस्यमयी पाताल लोक का रहस्य    इस भूभाग को प्राचीनकाल में प्रमुख रूप से  भागों में बांटा गया था इंद्रलोक पृथ्वी लोक और पाताल लोक इंद्रलोक हिमालय और उसके आसपास का क्षेत्र तथा आसमान तक पृथ्वी लोक अर्थात जहां भी जल जंगल और समतल भूमि रहने लायक है और पाताल लोक अर्थात रेगिस्तान और समुद्र के किनारे के अलावा समुद्र के अंदर के लोक पाताल लोक भी  प्रकार के बताए गए हैं जब हम यह कहते हैं कि भगवान विष्णु ने राजा बलि को पाताल लोक का राजा बना दिया था तो किस पाताल का यह जानना भी जरूरी है  पातालों में से एक पाताल का नाम पाताल ही है  कौन रहता है पाताल में    हिन्दू धर्म में पाताल लोक की स्थिति पृथ्वी के नीचे बताई गई है नीचे से अर्थ समुद्र में या समुद्र के किनारे पाताल लोक में नाग दैत्य दानव और यक्ष रहते हैं राजा बालि को भगवान विष्णु ने पाताल के सुतल लोक का राजा बनाया है और वह तब तक राज करेगा जब तक कि कलियुग का अंत नहीं हो जाता राज करने के लिए किसी स्थूल शरीर की जरूरत नहीं होती सूक्ष्म शरीर से भी काम किया जा सकता है पुराणों के अनुसार राजा बलि अभी भी जीवित हैं और साल में एक बार पृथ्वी पर आते हैं प्रारंभि

In [20]:
count=[]
for sent in list_sents:
    count.append(len(tokenizer.tokenize(sent)))

#X_0 = np.array(list_sents,dtype='object')
# y_0 = np.array(annotated_df['one_fear_speech'])
# tokenizer = AutoTokenizer.from_pretrained(params['model_path'])
    
# X_train, X_train_length = encode_documents(X_0,params,tokenizer) 

In [27]:
count_greater=0
for ele in count:
    if(ele>128*30):
        count_greater+=1
count_greater

112

In [25]:
count

[236,
 85,
 392,
 42,
 1154,
 1245,
 2141,
 153,
 340,
 985,
 246,
 30,
 209,
 570,
 542,
 1067,
 837,
 594,
 869,
 1014,
 895,
 49,
 667,
 1035,
 3217,
 45,
 86,
 666,
 1395,
 311,
 33,
 137,
 627,
 93,
 52,
 210,
 5030,
 539,
 1452,
 3111,
 588,
 1052,
 86,
 65,
 44,
 478,
 43,
 1478,
 491,
 1261,
 269,
 2313,
 308,
 1163,
 1262,
 402,
 1441,
 322,
 883,
 143,
 265,
 415,
 494,
 459,
 1012,
 608,
 1423,
 1275,
 201,
 534,
 890,
 784,
 438,
 389,
 887,
 634,
 1419,
 123,
 311,
 135,
 1622,
 432,
 776,
 1189,
 805,
 876,
 819,
 250,
 480,
 428,
 946,
 832,
 884,
 85,
 197,
 44,
 138,
 1726,
 646,
 1715,
 323,
 3072,
 3307,
 202,
 60,
 645,
 1178,
 229,
 858,
 584,
 2242,
 2228,
 53,
 640,
 1551,
 264,
 596,
 221,
 2835,
 975,
 940,
 200,
 351,
 788,
 47,
 2251,
 74,
 450,
 1633,
 396,
 89,
 789,
 53,
 178,
 667,
 962,
 42,
 30,
 45,
 1084,
 945,
 81,
 1472,
 499,
 116,
 977,
 497,
 1244,
 640,
 1760,
 3063,
 854,
 907,
 2612,
 551,
 50,
 231,
 761,
 1118,
 53,
 37,
 461,
 4325,
 239,
 

In [ ]:
train_dataloader=return_dataloader(X_train,y_0,params,is_train=True)


In [ ]:
train_dataloader

In [ ]:
 for step, batch in tqdm(enumerate(train_dataloader)):

        b_input_ids = batch[0]
        b_labels = batch[1]
        break

In [ ]:
b_input_ids.shape

In [ ]:
b_input_ids[0][:20,1]